# Structural Optimization
- offshoot from the network flow notebook (will need definitions from there)

In [1]:
G = form_graph(5,5, .05, 1.4, seed=5)
show(G,size=5)

for n1,n2, data in G.edges(data=True):
    del data['Capacity']
    del data['Load']
    p1 = np.array(G.node[n1]['pos'])
    p2 = np.array(G.node[n2]['pos'])
    
    data['v'] = p1-p2
    
for n, data in G.nodes(data=True):
    del data['Flow']
    data['g'] = 0
    data['f'] = np.random.randn(2) - np.array([0,1])

NameError: name 'form_graph' is not defined

In [ ]:
strength = 3.0
max_A = 10

def reset_edge(G, keys):
    for key in keys:
        for n1,n2,data in G.edges(data=True):
            data[key] = 0
        
def reset_node(G, keys):   
    for key in keys:
        for n, data in G.nodes(data=True):
            data[key] = 0
            
def replace_val(G):
    for n, data in G.nodes(data=True):
        data['g'] = np.array(data['g'].value).flatten()
        
    for _,_,data in G.edges(data=True):
        for key in ('Area', 'Tension'):
            data[key] = data[key].value

def struct(G, fixed):
    reset_node(G, ('g',))
    
    
    constr = []
    obj = 0
    
    for n1,n2,data in G.edges(data=True):
        t = cvx.Variable()
        a = cvx.Variable()
        data['Tension'] = t
        data['Area'] = a
        
        v = data['v']
        L = np.linalg.norm(v)

        constr += [cvx.abs(t) <= strength*a, a <= max_A] 
            
        G.node[n1]['g'] += v*t/L
        G.node[n2]['g'] -= v*t/L
        
        obj += L*a
        


    for n, data in G.nodes(data=True):
        if n in fixed:
            pass
        else:
            data['g'] += data['f']
            constr += [data['g'] == 0]


    prob = cvx.Problem(cvx.Minimize(obj), constr)
    prob.solve()
    replace_val(G)
    
    return prob.status

- solve structural opt problem with some fixed nodes

In [2]:
struct(G, [0,5,15,20])

NameError: name 'struct' is not defined

- display struct sizes

In [ ]:
show(G, edge_key='Area', labels=False)